<a href="https://colab.research.google.com/github/Afuu0503/ETL-Case-Study/blob/main/ETL_Case_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# books.toscrape.com Case Study ETL
## Python and beautifulsoup4 library

1. Data source: (https://books.toscrape.com/)
2. Data types: (thrillers and mystery books:  titles, prices, availability, product description, rating)
3. Checking the website regulations(robots.txt) ✔
4. Data Refresh Rate - Unfortunatelly in Google Colab it's not that easy to have data refreshed automatically. For this project it's not necessary to have data refreshed automatically

# Extracting data

In [1]:
!pip install requests beautifulsoup4 pandas

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_category(url, category_name):
    books = []
    page_url = url

    while True:
        response = requests.get(page_url)
        soup = BeautifulSoup(response.text, "html.parser")

        for book in soup.select("article.product_pod"):
            title = book.h3.a["title"]
            price = book.select_one("p.price_color").get_text(strip=True)
            availability = book.select_one("p.instock.availability").get_text(strip=True)
            rating = book.p["class"][1]

            detail_link = book.h3.a["href"].replace("../../../", "https://books.toscrape.com/catalogue/")

            detail_resp = requests.get(detail_link)
            detail_soup = BeautifulSoup(detail_resp.text, "html.parser")

            desc_header = detail_soup.find("div", id="product_description")
            if desc_header:
                product_description = desc_header.find_next_sibling("p").get_text(strip=True)
            else:
                product_description = "Brak opisu"

            books.append({
                "category": category_name,
                "title": title,
                "price": price,
                "availability": availability,
                "rating": rating,
                "description": product_description
            })

        next_page = soup.select_one("li.next > a")
        if next_page:
            page_url = url.rsplit("/", 1)[0] + "/" + next_page["href"]
        else:
            break

    return books


In [7]:
urls = {
    "Mystery": "https://books.toscrape.com/catalogue/category/books/mystery_3/index.html",
    "Thriller": "https://books.toscrape.com/catalogue/category/books/thriller_37/index.html"
}

all_books = []

for category, url in urls.items():
    print(f"Data load from: {category}")
    books = scrape_category(url, category)
    all_books.extend(books)

df = pd.DataFrame(all_books)
print(f"\n Loaded {len(df)} results.")
df.head()

Data load from: Mystery
Data load from: Thriller

 Loaded 43 results.


,category,title,price,availability,rating,description
0,Mystery,Sharp Objects,Â£47.82,In stock,Four,"WICKED above her hipbone, GIRL across her hear..."
1,Mystery,"In a Dark, Dark Wood",Â£19.63,In stock,One,"In a dark, dark wood Nora hasn't seen Clare fo..."
2,Mystery,The Past Never Ends,Â£56.50,In stock,Four,"A simple task, Attorney Chester Morgan thinks...."
3,Mystery,A Murder in Time,Â£16.64,In stock,One,"Beautiful and brilliant, Kendra Donovan is a r..."
4,Mystery,The Murder of Roger Ackroyd (Hercule Poirot #4),Â£44.10,In stock,Four,"In the village of King's Abbot, a widow's sudd..."


# Transforming the Data

In [8]:
df.shape[0]

43

In [9]:
df.shape[1]

6

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   category      43 non-null     object
 1   title         43 non-null     object
 2   price         43 non-null     object
 3   availability  43 non-null     object
 4   rating        43 non-null     object
 5   description   43 non-null     object
dtypes: object(6)
memory usage: 2.1+ KB


In [11]:
df.isnull().sum()

,0
category,0
title,0
price,0
availability,0
rating,0
description,0


In [12]:
df['price'] = df['price'].str.replace("Â£", '').astype(float)

In [13]:
df.head(43)

,category,title,price,availability,rating,description
0,Mystery,Sharp Objects,47.82,In stock,Four,"WICKED above her hipbone, GIRL across her hear..."
1,Mystery,"In a Dark, Dark Wood",19.63,In stock,One,"In a dark, dark wood Nora hasn't seen Clare fo..."
2,Mystery,The Past Never Ends,56.50,In stock,Four,"A simple task, Attorney Chester Morgan thinks...."
3,Mystery,A Murder in Time,16.64,In stock,One,"Beautiful and brilliant, Kendra Donovan is a r..."
4,Mystery,The Murder of Roger Ackroyd (Hercule Poirot #4),44.10,In stock,Four,"In the village of King's Abbot, a widow's sudd..."
5,Mystery,The Last Mile (Amos Decker #2),54.21,In stock,Two,In his #1 New York Times bestseller Memory Man...
6,Mystery,That Darkness (Gardiner and Renner #1),13.92,In stock,One,As a forensic investigator for the Cleveland P...
7,Mystery,Tastes Like Fear (DI Marnie Rome #3),10.69,In stock,One,Sarah Hilary won the 2015 Theakston's Crime No...
8,Mystery,A Time of Torment (Charlie Parker #14),48.35,In stock,Five,Jerome Burnel was once a hero. He intervened t...
9,Mystery,A Study in Scarlet (Sherlock Holmes #1),16.73,In stock,Two,In the debut of literature's most famous sleut...


In [14]:
df['category'].value_counts()

,count
category,
Mystery,32
Thriller,11


In [15]:
df.describe()

,price
count,43.000000
mean,31.646047
std,16.089497
min,10.690000
25%,17.525000
50%,25.400000
75%,45.960000
max,59.710000


In [18]:
#Duplicates only on title results
any_duplicates = df.duplicated(subset=['title']).any()

In [19]:
any_duplicates

np.False_

In [20]:
df.isnull().sum()

,0
category,0
title,0
price,0
availability,0
rating,0
description,0


# Data Load

In [21]:
df.to_csv("books.csv", index=False, encoding="utf-8")

# Visualization

Report is available in Looker platform under this link:
https://lookerstudio.google.com/reporting/088d8ad8-cd5b-4b31-bfd2-03624ef6d29c